# Kissasian Downloader

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
from IPython.display import clear_output
import threading
import os
import pandas as pd
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from openpyxl import load_workbook

## Please enter information below:

In [2]:
urlSeries = 'https://kissasian.lu/Drama/10-no-Himitsu'
seriesName = '10 no Himitsu'
downloadDIR = os.getcwd()+'\\out\\video'
pathToMine = os.getcwd()+'\\out\\Mine.xlsx'

In [3]:
# Temporary List Variables
episodeMark = []
linkTitles = []
mp4UploadLink = []
mp4UploadLinkForDownload = []
replace = []
# Trick
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}

#### Downloader magic function

In [4]:
from tqdm.notebook import tqdm
import requests
def download_file_mp4upload(url,n):
    if len(str(n))==1:
        temp_str = seriesName+' ep 00'+str(n)+'.mp4'        
    elif len(str(n))==2:
        temp_str = seriesName+' ep 0'+str(n)+'.mp4'
    else:
        temp_str = seriesName+' ep '+str(n)+'.mp4'
    # url = "http://www.ovh.net/files/10Mb.dat" #big file test
#     url = link
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
    
    # Streaming, so we can iterate over the response.
    response = requests.get(url,headers = headers, stream=True)
    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm( desc=temp_str,total=total_size_in_bytes, unit='iB', unit_scale=True)
    with open(temp_str, 'wb') as file:
        for data in response.iter_content(block_size):
#             print(response.status_code)
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")
        if (os.path.exists(str_path + '\\' + temp_str)):
            os.remove(str_path + '\\' + temp_str)
    if(response.status_code == 403):
        if (os.path.exists(str_path + '\\' + temp_str)):
            os.remove(str_path + '\\' + temp_str)
#         break
#     break

In [5]:
str_path = downloadDIR + '\\'+seriesName
isFirstRun = os.path.isdir(downloadDIR)
isPathExist = os.path.isdir(str_path)
check_file = False
resumeDownloadThread = False

In [6]:
if (not(isFirstRun)):
    os.mkdir(downloadDIR)

In [7]:
if (isPathExist):
    check_file = True
else:
    os.mkdir(str_path)

In [8]:
# prefs = {"download.default_directory" : downloadDIR}
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
# options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(options=options)  
urlHead9xB = 'http://9xbud.com/'

In [9]:
is_link_change = False
if (check_file):
    df = pd.read_excel(pathToMine,sheet_name=seriesName)
    for i in range(df.shape[0]):
        episodeMark.append(df['episode name'][i])
        linkTitles.append(df['Kissasian Link'][i])
        mp4UploadLink.append(df['mp4Upload Link'][i])
        mp4UploadLinkForDownload.append(df['9xbud Link'][i])
        if(os.path.exists(str_path+'\\'+episodeMark[i])):
            local_size = os.path.getsize(str_path+'\\'+episodeMark[i])
            response = requests.get(mp4UploadLinkForDownload[i],headers = headers, stream=True)
            print(response.status_code)
            if (response.status_code == 403):
                mp4UploadLinkForDownload.pop()
                is_link_change = True
                temp_link = urlHead9xB+mp4UploadLink[i][12:]
                driver.get(temp_link)
                time.sleep(10)
                temp_s = BeautifulSoup(driver.page_source,'lxml')
                temp_x_find_a = temp_s.find_all('a', attrs={'class':'max-w-sm mx-auto group block sm:inline-block border border-blue-500 px-6 w-full lg:w-auto lg:px-6 py-2 leading-4 text-blue-500 no-underline hover:border-blue-300 shadow'})
                mp4UploadLinkForDownload.append(temp_x_find_a[1].get('href'))    
                response = requests.get(mp4UploadLinkForDownload[i],headers = headers, stream=True)
            total_size_in_bytes= int(response.headers.get('content-length', 0))
            if(not(local_size==total_size_in_bytes)):
                resumeDownloadThread = True
                replace.append(i)
        else:
            replace.append(i)
            resumeDownloadThread = True    
            
        
        
# df.columns

In [10]:
driver.close()

In [11]:
# replace.append(4)

In [12]:
# df

In [14]:
# episodeMark[replace[0]]


In [15]:
if(is_link_change):
    data = {
        'episode name' : episodeMark,
        'Kissasian Link' : linkTitles,
        'mp4Upload Link' : mp4UploadLink,
        '9xbud Link' : mp4UploadLinkForDownload
    }
    df = pd.DataFrame(data)
    with pd.ExcelWriter(pathToMine, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:  
            df.to_excel(writer, sheet_name=seriesName)    

In [16]:
episode_number = len(mp4UploadLinkForDownload)

In [17]:
if(resumeDownloadThread):
    episode_number = len(replace)

In [18]:
# response = requests.get(mp4UploadLinkForDownload[i],headers = headers, stream=True)
# response.status_code == 403
# mp4UploadLink[0]
# temp_link = urlHead9xB+mp4UploadLink[i][12:]
# temp_link
# len(replace)
# resumeDownloadThread

In [19]:
if(not(resumeDownloadThread)):
#   Get the episode links:  
    page = requests.get(urlSeries)
    s = BeautifulSoup(page.text,'lxml')
    titles = s.find_all('li', attrs={'class':'episodeSub'})
    linkTitles = [title.a.get('href') for title in titles]
    linkTitles.reverse()
          

In [20]:
if(not(resumeDownloadThread)):
    n = 1
    episodeMark = [];
    for i in linkTitles:
        if len(str(n))==1:
            temp_str = seriesName+' ep 00'+str(n)+'.mp4'        
        elif len(str(n))==2:
            temp_str = seriesName+' ep 0'+str(n)+'.mp4'
        else:
            temp_str = seriesName+' ep '+str(n)+'.mp4'
        episodeMark.append(temp_str)
        n = n + 1

In [21]:
if(not(resumeDownloadThread)):
    # prefs = {"download.default_directory" : downloadDIR}
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    # options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(options=options)    

In [22]:
urlDomain = 'https://kissasian.lu'
urlMobile = '/Mobile/SwitchToMobile' 
urlHead9xB = 'http://9xbud.com/'

In [23]:
if(not(resumeDownloadThread)):
    driver.get(urlDomain+urlMobile)
    
    for link in linkTitles:
        driver.get(urlDomain+link+'&s=mp')
        temp_x = driver.page_source
        temp_s = BeautifulSoup(temp_x,'lxml')
        temp_x_find_a = temp_s.find_all('a')
        mp4UploadLink.append(temp_x_find_a[12].get('href'))
    

In [24]:
if(not(resumeDownloadThread)):
    for link in mp4UploadLink:
        temp_link = urlHead9xB+link[12:]
        driver.get(temp_link)
        time.sleep(10)
        temp_s = BeautifulSoup(driver.page_source,'lxml')
        temp_x_find_a = temp_s.find_all('a', attrs={'class':'max-w-sm mx-auto group block sm:inline-block border border-blue-500 px-6 w-full lg:w-auto lg:px-6 py-2 leading-4 text-blue-500 no-underline hover:border-blue-300 shadow'})
        mp4UploadLinkForDownload.append(temp_x_find_a[1].get('href'))    

In [28]:
if(not(resumeDownloadThread)):
    driver.close()
    data = {
        'episode name' : episodeMark,
        'Kissasian Link' : linkTitles,
        'mp4Upload Link' : mp4UploadLink,
        '9xbud Link' : mp4UploadLinkForDownload
    }
    df = pd.DataFrame(data)

In [29]:
# # isFirstRun = True
# pathToMine = os.getcwd()+'\\Mine.xlsx'
# resumeDownloadThread = False
# seriesName = "Check1"
# # os.getcwd()
# os.getcwd()+'\\Mine.xlsx'
# pathToMine
# os.path.exists(pathToMine)
# data = {
#         'episode name' : episodeMark,
# #         'Kissasian Link' : linkTitles,
# #         'mp4Upload Link' : mp4UploadLink,
#         '9xbud Link' : mp4UploadLinkForDownload
#     }
# # data
# pathToMine

In [30]:
if(not(resumeDownloadThread)):
    if(not(os.path.exists(pathToMine))):
#         book = load_workbook(pathToMine)
#         writer = pd.ExcelWriter(pathToMine, engine = 'openpyxl')
#         writer.book = book
        writer = pd.ExcelWriter(pathToMine, engine = 'xlsxwriter')
        df.to_excel(writer, sheet_name = seriesName)
        writer.close()        
#         print('if')
    else:
#         print('else')
        with pd.ExcelWriter(pathToMine, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:  
            df.to_excel(writer, sheet_name=seriesName)

In [31]:
sleep_count = 5
n = 1
m = 0
z = 0
os.chdir(str_path)
flagFinish = False
thread_one = threading.Thread()
thread_two = threading.Thread()
thread_three = threading.Thread()
# thread_four = threading.Thread()
# thread_five = threading.Thread()
# print(episode_fold_number)
# print(episode_fold_number)

while not(flagFinish):
    if(not(thread_one.is_alive())):
        if(resumeDownloadThread):
            m = replace[z]
            n = replace[z]+1
            z = z + 1
            if(z==episode_number):
                flagFinish = True
        thread_one = threading.Thread(target=download_file_mp4upload,args=(mp4UploadLinkForDownload[m],n))
        thread_one.start()
        m = m + 1
        n = n + 1
        if m == episode_number:
            flagFinish = True
    elif(not(thread_two.is_alive())):
        if(resumeDownloadThread):
            m = replace[z]
            n = replace[z]+1
            z = z + 1
            if(z==episode_number):
                flagFinish = True
        thread_two = threading.Thread(target=download_file_mp4upload,args=(mp4UploadLinkForDownload[m],n))
        thread_two.start()
        m = m + 1
        n = n + 1
        if m == episode_number:
            flagFinish = True
    elif(not(thread_three.is_alive())):
        if(resumeDownloadThread):
            m = replace[z]
            n = replace[z]+1
            z = z + 1
            if(z==episode_number):
                flagFinish = True
        thread_three = threading.Thread(target=download_file_mp4upload,args=(mp4UploadLinkForDownload[m],n))
        thread_three.start()
        m = m + 1
        n = n + 1
        if m == episode_number:
            flagFinish = True
#     elif(not(thread_four.is_alive())):
#         thread_four = threading.Thread(target=download_file_mp4upload,args=(mp4UploadLinkForDownload[m],n))
#         thread_four.start()
#         m = m + 1
#         n = n + 1
#         if m == episode_number:
#             flagFinish = True
#     elif(not(thread_five.is_alive())):
#         thread_five = threading.Thread(target=download_file_mp4upload,args=(mp4UploadLinkForDownload[m],n))
#         thread_five.start()
#         m = m + 1
#         n = n + 1
#         if m == episode_number:
#             flagFinish = True
    else:
        time.sleep(sleep_count)
time.sleep(sleep_count)

# print('complete execution')
    
    
# for i in range(episode_remaining_number):
#     print(str(n+i))
# #     print('hello')


10 no Himitsu ep 003.mp4:   0%|          | 0.00/492M [00:00<?, ?iB/s]

10 no Himitsu ep 001.mp4:   0%|          | 0.00/648M [00:00<?, ?iB/s]

10 no Himitsu ep 002.mp4:   0%|          | 0.00/459M [00:00<?, ?iB/s]

KeyboardInterrupt: 

In [ ]:
os.chdir('..')

In [ ]:
# os.getcwd()

In [ ]:
# dir(os.write)

In [ ]:
# mp4UploadLinkForDownload[4]

In [ ]:
# mp4UploadLink[4][12:]

In [ ]:
# driver.get(urlHead9xB+mp4UploadLink[4][12:])

In [ ]:
# temp_s = BeautifulSoup(driver.page_source,'lxml')
# temp_x_find_a = temp_s.find_all('a', attrs={'class':'max-w-sm mx-auto group block sm:inline-block border border-blue-500 px-6 w-full lg:w-auto lg:px-6 py-2 leading-4 text-blue-500 no-underline hover:border-blue-300 shadow'})
# # mp4UploadLinkForDownload[i] = (temp_x_find_a[1].get('href'))    
# # response = requests.get(mp4UploadLinkForDownload[i],headers = headers, stream=True)
# x = (temp_x_find_a[1].get('href'))    
# x

In [ ]:
# lista = []

In [ ]:
# lista.append("hello2")

In [ ]:
# lista.pop()

In [ ]:
# mp4UploadLinkForDownload[4] 

In [ ]:
# mp4UploadLinkForDownload[4] = x
# mp4UploadLinkForDownload[4] 